In [4]:
import sqlite3 as sql

In [28]:
import sqlite3 as sql


# create file for SQLite database we will make since it doesn’t exist)
conn = sqlite3.connect('/Users/a14806/Desktop/DNP Program/Fall 2025/Clinical Database Design/healthcare.db')
cursor = conn.cursor()

# Enable foreign key constraints
cursor.execute("PRAGMA foreign_keys = ON;")

# Drop existing tables if they exist (for clean recreation)
tables = [
    "PATIENT_PHONE",
    "PATIENT_SUBSTANCE",
    "SUBSTANCE",
    "TREATMENT",
    "PATIENT",
    "NURSE",
    "PHYSICIAN",
    "CASE_MANAGER"
]
for t in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {t};")

# Create tables
cursor.execute("""
CREATE TABLE CASE_MANAGER (
    CM_ID INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    birth_year INTEGER,
    gender TEXT
);
""")

cursor.execute("""
CREATE TABLE PHYSICIAN (
    DOC_ID INTEGER PRIMARY KEY,
    salary REAL,
    FOREIGN KEY (DOC_ID) REFERENCES CASE_MANAGER (CM_ID)
);
""")

cursor.execute("""
CREATE TABLE NURSE (
    NURSE_ID INTEGER PRIMARY KEY,
    hourly REAL,
    FOREIGN KEY (NURSE_ID) REFERENCES CASE_MANAGER (CM_ID)
);
""")

cursor.execute("""
CREATE TABLE PATIENT (
    MRN INTEGER PRIMARY KEY,
    birth_dt TEXT,
    prefer_lang TEXT,
    first_name TEXT,
    middle_name TEXT,
    last_name TEXT,
    maiden_name TEXT,
    address TEXT,
    race TEXT,
    ethnicity TEXT,
    CM_ID INTEGER,
    FOREIGN KEY (CM_ID) REFERENCES CASE_MANAGER (CM_ID)
);
""")

cursor.execute("""
CREATE TABLE TREATMENT (
    T_ID INTEGER PRIMARY KEY,
    type TEXT,
    description TEXT,
    adverse_effect TEXT,
    service_dt TEXT,
    MRN INTEGER,
    FOREIGN KEY (MRN) REFERENCES PATIENT (MRN)
);
""")

cursor.execute("""
CREATE TABLE SUBSTANCE (
    NAME TEXT PRIMARY KEY,
    alias TEXT,
    drug_code TEXT,
    coding_system TEXT
);
""")

cursor.execute("""
CREATE TABLE PATIENT_SUBSTANCE (
    MRN INTEGER,
    SUB_NAME TEXT,
    amount REAL,
    FOREIGN KEY (MRN) REFERENCES PATIENT (MRN),
    FOREIGN KEY (SUB_NAME) REFERENCES SUBSTANCE (NAME)
);
""")

cursor.execute("""
CREATE TABLE PATIENT_PHONE (
    MRN INTEGER,
    phone TEXT,
    FOREIGN KEY (MRN) REFERENCES PATIENT (MRN)
);
""")

# Commit and close connection
conn.commit()





In [29]:
#see if tables were created correctly
# Query all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# find and print them
tables = cursor.fetchall()
print("Tables in the database:")
for t in tables:
    print("-", t[0])



Tables in the database:
- CASE_MANAGER
- PHYSICIAN
- NURSE
- PATIENT
- TREATMENT
- SUBSTANCE
- PATIENT_SUBSTANCE
- PATIENT_PHONE


In [30]:
#check for schema as well for example of patient to verify it worked
table_name = "PATIENT"

# Use PRAGMA to get column info
cursor.execute(f"PRAGMA table_info({table_name});")
columns = cursor.fetchall()

# Print column details
print(f"Columns in table '{table_name}':")
for col in columns:
    cid, name, dtype, notnull, dflt_value, pk = col
    print(f"- {name} ({dtype})", end="")
    if pk:
        print(" [PRIMARY KEY]", end="")
    if notnull:
        print(" NOT NULL", end="")
    if dflt_value is not None:
        print(f" DEFAULT {dflt_value}", end="")
    print()  # new line


Columns in table 'PATIENT':
- MRN (INTEGER) [PRIMARY KEY]
- birth_dt (TEXT)
- prefer_lang (TEXT)
- first_name (TEXT)
- middle_name (TEXT)
- last_name (TEXT)
- maiden_name (TEXT)
- address (TEXT)
- race (TEXT)
- ethnicity (TEXT)
- CM_ID (INTEGER)


In [31]:
#insert random data generated 

cursor.executescript("""
INSERT INTO CASE_MANAGER (CM_ID, first_name, last_name, birth_year, gender) VALUES
(1, 'Sarah', 'Thompson', 1985, 'F'),
(2, 'David', 'Nguyen', 1978, 'M'),
(3, 'Emily', 'Brooks', 1990, 'F'),
(4, 'Michael', 'Patel', 1982, 'M'),
(5, 'Olivia', 'Johnson', 1988, 'F');

INSERT INTO PHYSICIAN (DOC_ID, salary) VALUES
(1, 185000.00),
(2, 223000.00),
(3, 199500.00),
(4, 245000.00),
(5, 210750.00);

INSERT INTO NURSE (NURSE_ID, hourly) VALUES
(1, 42.50),
(2, 38.75),
(3, 46.00),
(4, 40.25),
(5, 44.10);

INSERT INTO PATIENT (MRN, birth_dt, prefer_lang, first_name, middle_name, last_name, maiden_name,
                     address, race, ethnicity, CM_ID) VALUES
(101, '1989-03-12', 'English', 'Maria', 'L.', 'Garcia', 'Lopez', '215 Oak St, Denver CO', 'Hispanic', 'Hispanic', 1),
(102, '1972-11-02', 'Spanish', 'Carlos', 'J.', 'Ramirez', 'Torres', '98 Elm Dr, Dallas TX', 'Hispanic', 'Hispanic', 2),
(103, '1999-07-25', 'English', 'Lauren', 'M.', 'Davis', 'Carter', '453 Pine Rd, Seattle WA', 'White', 'Non-Hispanic', 3),
(104, '1958-05-19', 'English', 'Robert', 'T.', 'Kim', 'Choi', '72 Maple Ave, Chicago IL', 'Asian', 'Non-Hispanic', 4),
(105, '2002-12-08', 'French', 'Chloe', 'N.', 'Dupont', 'Martin', '14 Rue Clair, New Orleans LA', 'White', 'Non-Hispanic', 5);

INSERT INTO TREATMENT (T_ID, type, description, adverse_effect, service_dt, MRN) VALUES
(1, 'Medication', 'Administered antihypertensive therapy', 'None', '2024-08-12', 101),
(2, 'Surgery', 'Laparoscopic appendectomy', 'Nausea', '2024-09-03', 102),
(3, 'Therapy', 'Cognitive behavioral therapy session', 'Fatigue', '2024-10-14', 103),
(4, 'Diagnostic', 'MRI brain scan', 'None', '2024-07-22', 104),
(5, 'Medication', 'Insulin dose adjustment', 'Hypoglycemia', '2024-06-30', 105);

INSERT INTO SUBSTANCE (NAME, alias, drug_code, coding_system) VALUES
('Ibuprofen', 'Advil', 'DC101', 'ICD-10'),
('Acetaminophen', 'Tylenol', 'DC102', 'SNOMED'),
('Metformin', 'Glucophage', 'DC103', 'CPT'),
('Lisinopril', 'Prinivil', 'DC104', 'ICD-10'),
('Albuterol', 'Ventolin', 'DC105', 'SNOMED');

INSERT INTO PATIENT_SUBSTANCE (MRN, SUB_NAME, amount) VALUES
(101, 'Ibuprofen', 200),
(102, 'Metformin', 500),
(103, 'Lisinopril', 10),
(104, 'Acetaminophen', 650),
(105, 'Albuterol', 90);

INSERT INTO PATIENT_PHONE (MRN, phone) VALUES
(101, '(303) 555-1289'),
(102, '(214) 555-6722'),
(103, '(425) 555-9410'),
(104, '(312) 555-7734'),
(105, '(504) 555-3345');
""")

conn.commit()




In [ ]:
#now check that it did add this data by printing using print and 'SELECT' (retrieve the data)

print("\n--- CASE_MANAGER ---")
for row in cursor.execute("SELECT * FROM CASE_MANAGER;"):
    print(row)

print("\n--- PHYSICIAN ---")
for row in cursor.execute("SELECT * FROM PHYSICIAN;"):
    print(row)

print("\n--- NURSE ---")
for row in cursor.execute("SELECT * FROM NURSE;"):
    print(row)

print("\n--- PATIENT ---")
for row in cursor.execute("SELECT * FROM PATIENT;"):
    print(row)

print("\n--- TREATMENT ---")
for row in cursor.execute("SELECT * FROM TREATMENT;"):
    print(row)

print("\n--- SUBSTANCE ---")
for row in cursor.execute("SELECT * FROM SUBSTANCE;"):
    print(row)

print("\n--- PATIENT_SUBSTANCE ---")
for row in cursor.execute("SELECT * FROM PATIENT_SUBSTANCE;"):
    print(row)

print("\n--- PATIENT_PHONE ---")
for row in cursor.execute("SELECT * FROM PATIENT_PHONE;"):
    print(row)


--- CASE_MANAGER ---
(1, 'Sarah', 'Thompson', 1985, 'F')
(2, 'David', 'Nguyen', 1978, 'M')
(3, 'Emily', 'Brooks', 1990, 'F')
(4, 'Michael', 'Patel', 1982, 'M')
(5, 'Olivia', 'Johnson', 1988, 'F')

--- PHYSICIAN ---
(1, 185000.0)
(2, 223000.0)
(3, 199500.0)
(4, 245000.0)
(5, 210750.0)

--- NURSE ---
(1, 42.5)
(2, 38.75)
(3, 46.0)
(4, 40.25)
(5, 44.1)

--- PATIENT ---
(101, '1989-03-12', 'English', 'Maria', 'L.', 'Garcia', 'Lopez', '215 Oak St, Denver CO', 'Hispanic', 'Hispanic', 1)
(102, '1972-11-02', 'Spanish', 'Carlos', 'J.', 'Ramirez', 'Torres', '98 Elm Dr, Dallas TX', 'Hispanic', 'Hispanic', 2)
(103, '1999-07-25', 'English', 'Lauren', 'M.', 'Davis', 'Carter', '453 Pine Rd, Seattle WA', 'White', 'Non-Hispanic', 3)
(104, '1958-05-19', 'English', 'Robert', 'T.', 'Kim', 'Choi', '72 Maple Ave, Chicago IL', 'Asian', 'Non-Hispanic', 4)
(105, '2002-12-08', 'French', 'Chloe', 'N.', 'Dupont', 'Martin', '14 Rue Clair, New Orleans LA', 'White', 'Non-Hispanic', 5)

--- TREATMENT ---
(1, 'Medica

In [33]:
#now make it look super nice and pretty using pandas:)
import pandas as pd

tables = ["CASE_MANAGER", "PHYSICIAN", "NURSE", "PATIENT", "TREATMENT", "SUBSTANCE", "PATIENT_SUBSTANCE", "PATIENT_PHONE"]

for t in tables:
    df = pd.read_sql_query(f"SELECT * FROM {t};", conn)
    print(f"\n--- {t} ---")
    print(df)


--- CASE_MANAGER ---
   CM_ID first_name last_name  birth_year gender
0      1      Sarah  Thompson        1985      F
1      2      David    Nguyen        1978      M
2      3      Emily    Brooks        1990      F
3      4    Michael     Patel        1982      M
4      5     Olivia   Johnson        1988      F

--- PHYSICIAN ---
   DOC_ID    salary
0       1  185000.0
1       2  223000.0
2       3  199500.0
3       4  245000.0
4       5  210750.0

--- NURSE ---
   NURSE_ID  hourly
0         1   42.50
1         2   38.75
2         3   46.00
3         4   40.25
4         5   44.10

--- PATIENT ---
   MRN    birth_dt prefer_lang first_name middle_name last_name maiden_name  \
0  101  1989-03-12     English      Maria          L.    Garcia       Lopez   
1  102  1972-11-02     Spanish     Carlos          J.   Ramirez      Torres   
2  103  1999-07-25     English     Lauren          M.     Davis      Carter   
3  104  1958-05-19     English     Robert          T.       Kim        Choi  